Set global variables and paths for ensemble analysis 

In [1]:
#set global variables to pass into BASH commands. 
ctsmrepo='ctsmsept20'
defcase='SPdefault'
ens_directory='SPfates_ens'
caseroot='spfates_01_n'
paramsdir='param_files_sp'
ncases=6
defbuild='/glade/scratch/$USER/%defcase/bld/'

In [2]:
import os
notebookdr=os.getcwd() 

Go to the CTSM repo, create a deafult FATES 4x5 case.

In [3]:
%%bash -s "$ctsmrepo" "$defcase" 

ctsmrepo=$1
defcase=$2

#set compset and grid
reso=f45_f45_mg37 
comps=2000_DATM%GSWP3v1_CLM50%FATES_SICE_SOCN_MOSART_CISM2%NOEVOLVE_SWAV_SIAC_SESP

cd /glade/work/$USER/
#go to git directory
if [[ -d "git" ]]
then
cd git/
else
mkdir git
cd git
fi

#go to git repo, or checkout code
if [[ -d "$ctsmrepo" ]]
then
cd $ctsmrepo
else

#clone CTSM code
git clone https://github.com/escomp/ctsm $ctsmrepo
cd $ctsmrepo
./manage_externals/checkout_externals
cd src

#change CTSM code to SP branch. 
git remote add  rosie_ctsm_repo  https://github.com/rosiealice/ctsm
git fetch rosie_ctsm_repo
git checkout -b SP_api rosie_ctsm_repo/SP_api

#change FATES code to SP branch. 
cd fates
git remote add  rosie_fates_repo https://github.com/rosiealice/fates
git fetch rosie_fates_repo
git checkout -b fates_sp_sept rosie_fates_repo/fates_sp_sept

cd ../../
fi

cd cime/scripts
pwd
if [[ -d "$defcase" ]]
then
    echo "$defcase exists on your filesystem."
else
./create_newcase --case $defcase --compset 2000_DATM%GSWP3v1_CLM50%FATES_SICE_SOCN_MOSART_CISM2%NOEVOLVE_SWAV_SIAC_SESP --res f45_f45_mg37  --run-unsupported
./case.setup
./case.build
fi

cd $casename

/glade/work/rfisher/git/ctsmsept20/cime/scripts
SPdefault exists on your filesystem.


Adjust settings of default case. 

In [4]:
%%bash -s "$ctsmrepo" "$defcase" "$defbuild"

ctsmrepo=$1
defcase=$2
defbuild=$3

cd /glade/work/$USER/git/$ctsmrepo/cime/scripts/$defcase
pwd

echo 'updating settings',$newcase
./xmlchange BUILD_COMPLETE=TRUE;
./xmlchange EXEROOT=$defbuild;
./xmlchange CONTINUE_RUN=FALSE;
./xmlchange --id STOP_N --val 5;
./xmlchange --id STOP_OPTION --val nyears;
./xmlchange --id CLM_FORCE_COLDSTART --val on;


/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPdefault
updating settings,
For your changes to take effect, run:
./case.build --clean-all
./case.build
For your changes to take effect, run:
./case.build --clean-all
./case.build


Make ensemble of cases.

In [5]:
%%bash -s "$ctsmrepo" "$defcase" "$ens_directory" "$caseroot" $ncases

ctsmrepo=$1
defcase=$2
ens_directory=$3
caseroot=$4
ncases=$5

echo $ncases
echo $caseroot
echo $ens_directory
echo $ctsmrepo

cd /glade/work/$USER/git/$ctsmrepo/cime/scripts
pwd

if [[ -d "$ens_directory" ]]
then
echo "directory already exists"
else
mkdir $ens_directory
fi 

counter1=1
while [ $counter1 -le $ncases ]
do
  newcase=$caseroot$counter1
  rm -rf $newcase 
  if [ -d $ens_directory/$newcase ]
  then
    echo ' new case already exists',$ens_directory/$newcase   
  else
   echo 'making', $newcase
    ./create_clone --clone $defcase --case $ens_directory/$newcase ;
    ./case.setup;
  fi
    ((counter1++))
  done    


6
spfates_01_n
SPfates_ens
ctsmsept20
/glade/work/rfisher/git/ctsmsept20/cime/scripts
directory already exists
 new case already exists,SPfates_ens/spfates_01_n1
 new case already exists,SPfates_ens/spfates_01_n2
 new case already exists,SPfates_ens/spfates_01_n3
 new case already exists,SPfates_ens/spfates_01_n4
 new case already exists,SPfates_ens/spfates_01_n5
 new case already exists,SPfates_ens/spfates_01_n6


Fix settings in new cases


Make Default user_nl_clm File

In [6]:
%%bash -s "$ctsmrepo" "$defcase" "$notebookdr" "$paramsdir"

ctsmrepo=$1
echo $ctsmrepo
defcase=$2
notebookdr=$3
paramsdir=$4
echo $paramsdir
cd /glade/work/$USER/git/$ctsmrepo/cime/scripts
pwd
cp  $defcase/user_nl_clm $notebookdr/$paramsdir/user_nl_clm_defult
echo $notebookdr/$paramsdir/
cd $notebookdr/$paramsdir/

pftfilename_def=$notebookdr/$paramsdir

echo fates_use_sp=.true. >> user_nl_clm_defult
echo fates_use_nocomp=.true. >> user_nl_clm_defult
echo fates_use_fixed_biogeog=.true. >> user_nl_clm_defult
echo fates_paramfile=\'\pftfilename_default\' >> user_nl_clm_defult

cat user_nl_clm_defult


ctsmsept20
param_files_sp
/glade/work/rfisher/git/ctsmsept20/cime/scripts
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp/
!----------------------------------------------------------------------------------
! Users should add all user specific namelist changes below in the form of 
! namelist_var = new_namelist_value 
!
! EXCEPTIONS: 
! Set use_cndv           by the compset you use and the CLM_BLDNML_OPTS -dynamic_vegetation setting
! Set use_vichydro       by the compset you use and the CLM_BLDNML_OPTS -vichydro           setting
! Set use_cn             by the compset you use and CLM_BLDNML_OPTS -bgc  setting
! Set use_crop           by the compset you use and CLM_BLDNML_OPTS -crop setting
! Set spinup_state       by the CLM_BLDNML_OPTS -bgc_spinup      setting
! Set co2_ppmv           with CCSM_CO2_PPMV                      option
! Set dtime              with L_NCPL                             option
! Set fatmlndfrc         with LND_DOMAIN_PATH/LND_DOMAIN_FILE    options
!

Get working dir from python

Make a directory for the parameter files to go in and copy default .nc parameter file there. 

In [7]:
%%bash -s "$ctsmrepo" "$notebookdr"  "$paramsdir"
echo $1
ctsmrepo=$1
notebookdr=$2
paramsdir=$3
echo $ctsmrepo

cd $notebookdr
if [[ -d "$paramsdir" ]]
then
   cd $paramsdir
   echo "existing param firectory"
   ls -l
else 
    mkdir $paramsdir
    echo "made param firectory"
fi
pwd

cp /glade/work/$USER/git/$ctsmrepo/src/fates/parameter_files/fates_params_default.nc fates_params_default.nc
ls -l


ctsmsept20
ctsmsept20
existing param firectory
total 1665
-rw-r--r-- 1 rfisher ncar 64559 Nov 10 04:13 fates_params_default.cdl
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 05:57 fates_params_default.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:08 fates_params_update.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_1.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:16 fates_params_update_10.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_2.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_3.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_4.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_5.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_6.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_7.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_8.nc
-rw-r--r-- 1 rfisher ncar 61944 Nov 10 04:17 fates_params_update_9.nc
-rw-r--r-- 1 rfisher ncar  1529 No

In [8]:
%%bash -s "$ctsmrepo" "$notebookdr" "$paramsdir" 

ctsmrepo=$1
echo $1



ctsmsept20


Create .nc version of current defualt file. I FAILED ATT THIS. NEEDS INPUT..

Mpdify default parameter file

In [10]:
import netCDF4 as nc4
import sys
import os
import shutil

def copy_clobber(filename1, filename2):
    try:
        os.remove(filename2)
        print('replacing file: '+filename2)
    except:
        print('file does not yet exist: '+filename2)
    shutil.copyfile(filename1, filename2)

filename_template = notebookdr+'/'+paramsdir+'/fates_params_default.nc'
   
for i in range(1,10):
    filename_out= notebookdr+'/'+param_files_sp+/fates_params_update_'+str(i)+'.nc'
    print(filename_out)
    copy_clobber(filename_template,filename_out)

    fin = nc4.Dataset(filename_out, 'r+')

    fates_leaf_vcmax25top_var = fin.variables['fates_leaf_vcmax25top']
    fates_leaf_vcmax25top_var[:] = i*10
    fin.close()


SyntaxError: invalid syntax (<ipython-input-10-c543714914ce>, line 17)

In [ ]:
%%shell -s "$defparamfile_cdl" "$defparamfile_nc" "$ctsmrepo"
module load nco

defparamfile_cdl=$1
defparamfile_nc=$2
ctsmrepo=$3

cd /glade/work/$USER/git/$ctsmrepo/src/fates/parameter_files

ncgen -o defparamfile_nc defparamfile_cdl

Make ensemble of parameer files

Point each ensemble script at different parameter file

In [ ]:
%%bash
defcase=SPdefault
directory=SPfates_ens
caseroot=spfates_01_n
ncases=6
ctsmrepo=ctsmsept20
defbuild=/glade/scratch/$USER/%defcase/bld/

cd /glade/work/$USER/git/$ctsmrepo/cime/scripts
counter1=1
cd $directory
pwd
ls -l
while [ $counter1 -le $ncases ]
do
  echo $counter1
  newcase=$caseroot$counter1
  echo $newcase

  if [ -d $newcase ]
  then
   cd $newcase
    echo 'updating parameter file',$newcase

   # sed -i 's/6PFTS/'$pftfilename'/g' user_nl_clm
   # cat user_nl_clm

    cd ../
  else
   echo 'no case', $newcase
  fi

  ((counter1++))
  done


Submit ensemble of cases

In [ ]:
%%bash
directory=SPfates_ens
caseroot=spfates_01_n
ncases=6
ctsmrepo=ctsmsept20

cd /glade/work/$USER/git/$ctsmrepo/cime/scripts

counter1=1
cd $directory
pwd
ls -l
while [ $counter1 -le $ncases ]
do
  echo $counter1
  newcase=$caseroot$counter1
  echo $newcase

  if [ -d $newcase ]
  then
    cd $newcase
    echo 'submitting job',$newcase
    ./case.submit;
    cd ../
  else
   echo 'no case', $newcase
  fi

  ((counter1++))
  done